In [1]:
#!git clone https://github.com/noahcao/OC_SORT.git
!git clone https://github.com/JonathonLuiten/TrackEval.git

Cloning into 'TrackEval'...


In [4]:
!wget https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
!unzip data.zip -d TrackEval

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [ ]:
!pip install natsort

In [ ]:
#!pip install -r TrackEval/minimum_requirements.txt

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2507 lines of output]
      Ignoring numpy: markers 'python_version == "3.5" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment


     --------------------------------------- 24.6/24.6 MB 13.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


      Ignoring numpy: markers 'python_version == "3.5" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
        Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
        Using cached setuptools-67.6.1-py3-none-any.whl (1.1 MB)
           -------------------------------------- 988.1/988.1 kB 7.8 MB/s eta 0:00:00
           ---------------------------------------- 6.4/6.4 MB 20.4 MB/s eta 0:00:00
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
           ------------------------------------- 222.3/222.3 kB 13.3 MB/s eta 0:00:00
        error: subprocess-exited-with-error
     


              adding 'build\src.win-amd64-3.1\numpy\core\src\common' to include_dirs.
            creating build\src.win-amd64-3.1\numpy\core\src\npysort
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\quicksort.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\mergesort.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\timsort.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\heapsort.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\radixsort.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\common\npy_partition.h
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\selection.c
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\common\npy_binsearch.h
            conv_template:> build\src.win-amd64-3.1\numpy\core\src\npysort\binsearch.c
            None - nothing done with h_files = ['build\\src

In [5]:
!pip install psutil
!pip install seaborn


     ------------------------------------ 293.3/293.3 kB 452.9 kB/s eta 0:00:00


In [18]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\pierr/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirements "torchvision>=0.8.1" "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 2 packages updated per C:\Users\pierr\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-4-3 Python-3.10.0 torch-2.0.0+cpu CPU



In [16]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\pierr\\appdata\\local\\programs\\python\\python310\\lib\\site-packages\\protobuf-3.20.3-py3.10-nspkg.pth'



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.10.1 requires protobuf<5.0.0dev,>=3.20.1, but you have protobuf 3.19.6 which is incompatible.


In [14]:
torch.__version__

'2.0.0+cpu'

In [ ]:
imgs = ['https://ultralytics.com/images/zidane.jpg']  # batch of images

# Inference
results = model(imgs)

In [ ]:
!pip install natsort

In [11]:
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import numpy as np

from OC_SORT.trackers.ocsort_tracker.ocsort import OCSort
from dataloader import dataloader

In [8]:

from OC_SORT.trackers.ocsort_tracker.ocsort import OCSort

In [26]:
from dataloader import dataloader

### Etablissement du modèle

In [9]:
tracker = OCSort(0.5)

### Récupération des résultats pour les différents dataset public

### Calcul de la performance pour les différents dataset public

In [18]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT17 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL MPNTrack --METRICS HOTA --USE_PARALLEL False --NUM_PARALLEL_CORES 1

Error importing BURST due to missing underlying dependency: No module named 'pycocotools'

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : d:\POLYM1\TrimestreHiver2023\INF6804\INF6804\TP3\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER       

In [ ]:
image=cv2.imread("frame0.jpg")
results = model(image)
results.print()
results.save()

In [25]:
model = YoloHub(
    config_path="./default_config.yaml",
    model_type="yolov5",
    model_path="yolov5x6.pt",
)

result = model.predict(source="./frames/frame0.jpg")

YOLOv5  2023-3-31 Python-3.8.16 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 270 layers, 7235389 parameters, 0 gradients
Adding AutoShape... 
  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [5]:


tracker = OCSort(0.4)
#dets = yl.detector(image)
online_targets = tracker.update(dets)

NameError: name 'dets' is not defined

### Récupération des résultats pour la vidéo du tp

In [12]:
dl=dataloader("./frames/","./results.txt")
for i in range(364,430):
  image=dl.load_input(i)
  img_size=np.shape(image)
  results = model(image)
  res=results.pandas().xyxy[0][['xmin','ymin','xmax','ymax','confidence','class']]
  cup_results=res[res['class']==41]
  np_res=cup_results.to_numpy()
  tf_res=tf.constant(np_res)
  online_targets = tracker.update(tf_res,img_size,img_size)
  dl.load_result(i,online_targets)
dl.f.close()

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
